# Model of work distribution in a flat organization

* no managers; only workers
* no support staff; all infrastructure for communication and tasks works without failure
* no dark patterns

TODO:
* use realistic distributions (e.g., power law) instead of uniform
* instead of searching randomly, search second-order contacts

In [ ]:
import random
import numpy
print("numpy", numpy.__version__)
import pandas
print("pandas", pandas.__version__)
import seaborn
print("seaborn", seaborn.__version__)
import matplotlib
print("matplotlib", matplotlib.__version__)
from matplotlib import pyplot as plt
import sys
print(sys.version_info)

In [ ]:
pandas.set_option('display.max_rows', None)
pandas.set_option('display.max_columns', None)

### global variables for simulation

#### user-defined

In [ ]:
skill_set_for_people = ['A','B'] # ['A','B','C'] # uniform distribution
skill_set_for_tasks  = ['A','B'] # ['A','B','C'] # uniform distribution
max_skill_level_per_person      = 3  # uniform distribution
max_skill_level_per_task        = 3  # uniform distribution
max_number_of_tasks_per_process = 6  # uniform distribution
max_task_duration_in_ticks      = 1  # uniform distribution

number_of_people = 10
social_circle_size = 2

max_ticks_to_simulate = 100

#### not set by user

In [ ]:
process_id = 0

### input validation

In [ ]:
if len(skill_set_for_people)<1:
    raise Exception("invalid value")
if len(skill_set_for_tasks)<1:
    raise Exception("invalid value")
    
if max_skill_level_per_person<1:
    raise Exception("invalid value")
    
if max_skill_level_per_task<1:
    raise Exception("invalid value")
if max_number_of_tasks_per_process<1:
    raise Exception("invalid value")
if max_task_duration_in_ticks<1:
    raise Exception("invalid value")
if number_of_people<1:
    raise Exception("invalid value")
if social_circle_size<0:
    raise Exception("invalid value")
if max_ticks_to_simulate<1:
    raise Exception("invalid value")

### The two classes: person and process

In [ ]:
class CreatePerson():
    # static variables
#    skill_set = ['A','B','C']
#    max_skill_level = 3
    
    def __init__(self, unique_id:int):
        """
        each person has a
        * uniquie_id
        * skill/specialization matrix
        """
        self.unique_id = unique_id # problem: this can be set by the user after initialization; see https://stackoverflow.com/a/48709694/1164295
        self.backlog_of_processes = []
        self.status = "idle"
        self.assigned_process = None 
        self.contact_list = None
        self.skill_specialization_dict = {}
        self.work_journal_per_tick = {}
        for skill in skill_set_for_people:
            self.skill_specialization_dict[skill] = random.randint(0,max_skill_level_per_person)
        return
        
    def set_status(self, status):
        """
        status is a string with value idle xor working xor coordinating
        
        As this class is currently written, user can set "status" to arbitrary values
        """
        try:
            assert(status=="idle" or 
                   status=="working" or 
                   status=="coordinating")
            self.status = status
            return "status set to '"+status+"'"
        except AssertionError:
            return "ERROR: failed to set status; use either 'idle' xor 'working' xor 'coordinating'"
        return "ERROR: failed"
        

    def add_person_to_contact_list(self, person_id):
        """
        https://en.wikipedia.org/wiki/Dunbar%27s_number
        """
        self.contact_list.append(person_id)
        if len(self.contact_list)>social_circle_size:
            self.contact_list.pop(0)
        return
    
    #def add_task_to_backlog(self, work_item):
    #    self.backlog_of_processes.append(work_item)
    #    return 
        
    #def get_next_task_from_backlog(self):
    #    return self.backlog_of_processes.pop(0)
    
    #def show_skill_specialization(self):
    #    return self.skill_specialization_dict
        
    #def show_task_backlog(self):
    #    return self.backlog_of_processes

In [ ]:
new_person = CreatePerson(1)
new_person.set_status("asdf")

In [ ]:
new_person.set_status("working")

In [ ]:
list_of_people = []
for person_index in range(number_of_people):
    #this_person_dict = {}
    #for specialization in skill_set_for_people:
    #    this_person_dict[specialization] = random.choice(range(max_skill_level_per_person))
    
    list_of_people.append(CreatePerson(person_index))

In [ ]:
for person in list_of_people:
    print(person.unique_id)

In [ ]:
max_skill_per_specialization = {}

# initialize to -1
for specialization in skill_set_for_people:
    max_skill_per_specialization[specialization] = -1
    
for person in list_of_people:
    #print(person.skill_specialization_dict)
    for persons_specialization, persons_skilllevel in person.skill_specialization_dict.items():
        if persons_skilllevel>max_skill_per_specialization[persons_specialization]:
            max_skill_per_specialization[persons_specialization] = persons_skilllevel
            
for specialization, skilllevel in max_skill_per_specialization.items():
    if skilllevel<max_skill_level_per_person:
        print("WARNING: population lacks max skill-level for",specialization)
        print("As a consquence, some tasks cannot be completed by this population")

In [ ]:
class CreateProcess():
    """
    processes are a sequence of tasks
    """
    # static variables
    #skill_set = ['A','B','C']
    #max_skill_level = 3
    
    
    def __init__(self, unique_id:int):
        self.unique_id = unique_id
        self.list_of_tasks = []
        # a process is a sequence of tasks
#        for task_id in range(process.max_number_of_tasks):
        for task_id in range(random.randint(1,max_number_of_tasks_per_process)):
            #task_dict = {"task id": task_id}
            # task is either completed, active, or waiting
            #task_dict['status'] = "waiting"
            # each task has a skill, specialization, duration
            duration = random.randint(1,max_task_duration_in_ticks)
            task_dict = {'process ID': unique_id,
                         "task id": task_id,
                         'specialization': random.choice(skill_set_for_tasks),
                         'skill level': random.randint(1,max_skill_level_per_task),
                         'total duration': duration,
                         'remaining duration': duration}
            self.list_of_tasks.append(task_dict)
   
    def show_tasks(self):
        for task_dict in self.list_of_tasks:
            print(task_dict)
        return
    
    #def show_process(self):
    #    return self.list_of_tasks
    
    #def start_next_task(self):
    #    """
    #    removes task from process
    #    """
    #    return self.list_of_tasks.pop()
    
    #def what_is_next_task(self):
    #    return self.list_of_tasks[-1]

In [ ]:
new_p = CreateProcess(101)
new_p.list_of_tasks

instead of a static list of processes to draw from,   
create an infinite backlog of work

In [ ]:
def new_process():
    global process_id
    process_id+=1
    return CreateProcess(process_id)

### Assessment tools

In [ ]:
def show_all_people(list_of_people):
    for person in list_of_people:
        print('id=',person.unique_id, 
              '; skill matrix=',person.skill_specialization_dict)

In [ ]:
show_all_people(list_of_people)

In [ ]:
def current_status_of_people(list_of_people):
    """
    what is each person doing?
    """
    for person in list_of_people:
        try:
            task = person.assigned_process.list_of_tasks[0]
        except AttributeError:
            task = None
        print('person id=',person.unique_id, 
                  '; status=',person.status,
                  '; task=',task,
                  '; number of processes in backlog=',len(person.backlog_of_processes))


In [ ]:
current_status_of_people(list_of_people)

In [ ]:
def cumulative_task_backlog_size(list_of_people):
    backlog_count = 0
    for person in list_of_people:
        backlog_count += len(person.backlog_of_processes)
    return backlog_count

In [ ]:
cumulative_task_backlog_size(list_of_people)

In [ ]:
def all_idle(list_of_people):
    for person in list_of_people:
        if person.status != "idle":
            return False
    return True

In [ ]:
all_idle(list_of_people)

### Initialize Simulation

In [ ]:
# randomly distribute processes to people

list_of_processes = [] # track the generated processes

for index in range(len(list_of_people)):
    #print('person ID',list_of_people[index].unique_id)
    a_process = new_process()
    list_of_processes.append(a_process)
    try:
        list_of_people[index].backlog_of_processes.append(a_process)
    except IndexError:
        print("there are more people than processes")
        break

In [ ]:
current_status_of_people(list_of_people)

### Begin ticks of Simulation

In [ ]:
def pick_a_random_person(person_index, contacts, list_of_people):
    """
    find someone who is not myself and is not someone I already know
    """
    attempts = 0
    try:
        len(contacts)
    except TypeError: # contacts is None
        contacts=[]
    while (attempts<100):
        another_person = random.choice(range(len(list_of_people)))
        if ((another_person not in contacts) and 
            (another_person != person_index)):
            return another_person
    print("failed to find another person who is not a contact")
    return None
    

In [ ]:
def simulate(max_ticks_to_simulate: int,
             list_of_people: list):
    """
    """
    tick=-1
    while ((tick<max_ticks_to_simulate)):
        tick=tick+1
        print('\n===== tick',tick,'=====')
        print("   ===== status at the leading edge of this tick: =====")
        current_status_of_people(list_of_people)
        print("   ===== updates happening in this tick: =====")
        # each person looks in their backlog for work
        for person_index in range(len(list_of_people)):
            skill_dict = list_of_people[person_index].skill_specialization_dict
            print("person",person_index,'has skills',skill_dict)

            # initialize the work journal        
            list_of_people[person_index].work_journal_per_tick[tick] = {'number of processes in backlog=': 
                                                                        len(list_of_people[person_index].backlog_of_processes)}
            print("   person",person_index,"has backlog length",list_of_people[person_index].backlog_of_processes)

            if list_of_people[person_index].status=="idle":
                assert(list_of_people[person_index].assigned_process is None)
                try:
                    # new processes get appended, so the oldest task is at position 0
                    list_of_people[person_index].assigned_process = list_of_people[person_index].backlog_of_processes.pop(0)
                    print("   person",person_index,"got task from their backlog")
                    print("      process consists of",list_of_people[person_index].assigned_process.list_of_tasks)
                    list_of_people[person_index].work_journal_per_tick[tick]['task from'] = 'my own backlog'
                except IndexError: # pop from empty list
                    a_process = new_process()
                    list_of_processes.append(a_process) # save process to list for post-simulation analysis
                    list_of_people[person_index].assigned_process = a_process
                    print("   person",person_index,"had no task in backlog; got new process from infinite queue")
                    print("      process consists of",list_of_people[person_index].assigned_process.list_of_tasks)
                    list_of_people[person_index].work_journal_per_tick[tick]['task from'] = 'infinite backlog'

                # can the person do the task, or do they need to coordinate?
                if (list_of_people[person_index].assigned_process.list_of_tasks[0]['skill level'] <= 
                    skill_dict[list_of_people[person_index].assigned_process.list_of_tasks[0]['specialization']]): # person can do task
                    print("   person",person_index,"can do the task!")
                    list_of_people[person_index].status="working"
                    list_of_people[person_index].work_journal_per_tick[tick]['status was'] = "idle"
                    list_of_people[person_index].work_journal_per_tick[tick]['status is now'] = "working"
                else: # person doesn't have sufficient skill for task; person needs to find someone else
                    print("   person",person_index,"does not have sufficient skill")
                    list_of_people[person_index].status="coordinating"
                    list_of_people[person_index].work_journal_per_tick[tick]['status was'] = "idle"
                    list_of_people[person_index].work_journal_per_tick[tick]['status is now'] = "coordinating"

            # at this point, regardless of status, the person has a task
            if not list_of_people[person_index].assigned_process:
                raise Exception("person",person_index,"should have a task")

            my_process = list_of_people[person_index].assigned_process
            print("   person",person_index,"has an assigned task",my_process.list_of_tasks[0])
            list_of_people[person_index].work_journal_per_tick[tick]['task'] = my_process.list_of_tasks[0]

            assert((list_of_people[person_index].status=="working") or 
                   (list_of_people[person_index].status=="coordinating"))

            if list_of_people[person_index].status=="working":
                list_of_people[person_index].work_journal_per_tick[tick]['status was'] = "working"
                speedup = skill_dict[my_process.list_of_tasks[0]['specialization']]/my_process.list_of_tasks[0]['skill level']
                print("   speedup for person",person_index,"is", speedup)
                my_process.list_of_tasks[0]['remaining duration'] = my_process.list_of_tasks[0]['remaining duration'] - speedup # "doing the work"
                if (my_process.list_of_tasks[0]['remaining duration']<=0): # task was completed
                    print("   task completed!")
                    list_of_people[person_index].assigned_process = None
                    list_of_people[person_index].status = "idle"
                    list_of_people[person_index].work_journal_per_tick[tick]['outcome'] = "task completed"
                    list_of_people[person_index].work_journal_per_tick[tick]['status is now'] = "idle"
                else: # update the task to reflect there being less work remaining because the person did some work
                    list_of_people[person_index].assigned_process = my_process
                    list_of_people[person_index].work_journal_per_tick[tick]['outcome'] = "worked but task remains"
                    list_of_people[person_index].work_journal_per_tick[tick]['status is now'] = "working"

            if list_of_people[person_index].status=="coordinating":
                list_of_people[person_index].work_journal_per_tick[tick]['status was'] = "coordinating"
                contacts = list_of_people[person_index].contact_list
                print("   person",person_index,"contacts=",contacts)
                if contacts: # I know people!
                    print("   person",person_index,"looks in contact list")
                    for contact_id in contacts: # do the people I know have the skills to do this task?
                        contacts_skill_dict = list_of_people[contact_id].skill_specialization_dict
                        if (my_process.list_of_tasks[0]['skill level'] <= contacts_skill_dict[my_process.list_of_tasks[0]['specialization']]): # contact can do task
                            list_of_people[contact_id].backlog_of_processes.append(process)
                            list_of_people[person_index].assigned_process = None
                            list_of_people[person_index].status = "idle"
                            print("   person",person_index,"gave task to person",contact_id,"from contact list")
                            list_of_people[person_index].work_journal_per_tick[tick]['outcome'] = "gave task to person "+str(contact_id)+" from contact list"
                            list_of_people[person_index].add_person_to_contact_list(contact_id)
                            print("   person",person_index,"added",contact_id,"to list of contacts")
                            list_of_people[person_index].work_journal_per_tick[tick]['status is now'] = "idle"
                # after looking through contacts, if the status is still "coordinating", 
                # then person didn't have a contact who could do the work
                if list_of_people[person_index].status=="coordinating": 
                    another_person_id = pick_a_random_person(person_index, contacts, list_of_people)
                    another_person_skill_dict = list_of_people[another_person_id].skill_specialization_dict
                    if (my_process.list_of_tasks[0]['skill level'] <= another_person_skill_dict[my_process.list_of_tasks[0]['specialization']]): # another_person can do task
                        list_of_people[another_person_id].backlog_of_processes.append(my_process)
                        list_of_people[person_index].assigned_process = None
                        list_of_people[person_index].status = "idle"
                        print("   person",person_index,"gave task to person",another_person_id,"from random search")
                        list_of_people[person_index].work_journal_per_tick[tick]['outcome'] = "gave task to person "+str(another_person_id)+" from random search"
                        list_of_people[person_index].work_journal_per_tick[tick]['status is now'] = "idle"
                    else:
                        print("   person",person_index,"not able give to",another_person_id,"from random search")
                        list_of_people[person_index].work_journal_per_tick[tick]['outcome'] = "not able to give to "+str(another_person_id)+" from random search"
                        list_of_people[person_index].work_journal_per_tick[tick]['status is now'] = "coordinating"

    return list_of_people

In [ ]:
list_of_people = simulate(max_ticks_to_simulate,list_of_people)

In [ ]:
list_of_processes[0].list_of_tasks

## Post-simulation analysis

### a single person

In [ ]:
list_of_dicts = []
for tick_index, journal_dict in list_of_people[0].work_journal_per_tick.items():
    #print(tick_index)
    this_dict = {'tick': tick_index}
    for k,v in journal_dict.items():
        if k == "task":
            for task_k, task_v in v.items():
                this_dict[task_k] = task_v
        else:    
            this_dict[k] = v
    #print(this_dict)
    list_of_dicts.append(this_dict)

In [ ]:
list_of_people[0].skill_specialization_dict

In [ ]:
df_a_persons_journal = pandas.DataFrame(list_of_dicts)
df_a_persons_journal

# Visualization of the Simulation

In [ ]:
for index,person in enumerate(list_of_people):
    print(index,person.skill_specialization_dict)

In [ ]:
# TODO: A heat map of the ensemble of people for specializations and skill levels

In [ ]:
aggregate_person_dict = {}
for specialization in skill_set_for_people:
    aggregate_person_dict[specialization] = [0 for _ in range(max_skill_level_per_person)]

#aggregate_person_dict

In [ ]:
for person_id, person in enumerate(list_of_people):
    for specialization, skilllevel in person.skill_specialization_dict.items():
        #print(person_id, specialization, skilllevel)
        if skilllevel>0:
            aggregate_person_dict[specialization][skilllevel-1] += 1
        #print(aggregate_person_dict)
        
#aggregate_person_dict

In [ ]:
df_cumulative_people = pandas.DataFrame(aggregate_person_dict)
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.set_index.html
df_cumulative_people.set_index(pandas.Index(list(range(1,max_skill_level_per_person+1))))
df_cumulative_people

In [ ]:
# https://stackoverflow.com/a/34712544/1164295
seaborn.set(font_scale=1.5)
# https://stackoverflow.com/a/57266083/1164295
seaborn.heatmap(df_cumulative_people, 
                annot=True, 
                fmt="d",
                yticklabels=df_cumulative_people.index, 
                cmap='Blues', 
                linewidths=1, 
                linecolor='black');
plt.title("Cumulative counts across all people")
plt.gca().set_xlabel('Specialization', fontsize=15);
plt.gca().set_ylabel('Skill-level', fontsize=15);

In [ ]:
# TODO: A heat map of the ensemble of task per specializations in skill levels

In [ ]:
list_of_processes[0].list_of_tasks[0]

In [ ]:
aggregate_task_dict = {}
for specialization in skill_set_for_tasks:
    aggregate_task_dict[specialization] = [0 for _ in range(max_skill_level_per_task)]

aggregate_task_dict

In [ ]:
for process in list_of_processes:
    for task in process.list_of_tasks:
        #print('specialization:',task['specialization'])
        #print('skill-level:',task['skill level'])
        if task['skill level']>0:
            aggregate_task_dict[task['specialization']][task['skill level']-1] += 1

In [ ]:
df_cumulative_tasks = pandas.DataFrame(aggregate_task_dict)
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.set_index.html
df_cumulative_tasks = df_cumulative_tasks.set_index(pandas.Index(list(range(1,max_skill_level_per_task+1))))

In [ ]:
# https://seaborn.pydata.org/generated/seaborn.heatmap.html

# https://matplotlib.org/stable/gallery/images_contours_and_fields/image_annotated_heatmap.html

# https://stackoverflow.com/a/34712544/1164295
seaborn.set(font_scale=1.5)
# https://stackoverflow.com/a/57266083/1164295
seaborn.heatmap(df_cumulative_tasks, 
                annot=True, 
                fmt="d",
                yticklabels=df_cumulative_tasks.index, 
                cmap='Blues', 
                linewidths=1, 
                linecolor='black');
plt.title("Cumulative counts across all tasks")
plt.gca().set_xlabel('Specialization', fontsize=15);
plt.gca().set_ylabel('Skill-level', fontsize=15);

In [ ]:
# TODO: Heat map with axes of time and person index, 
# with the color being the length of the backlog per person per time

In [ ]:
person_time_backlog_dict = {}
for person_index in range(len(list_of_people)):
    
    description=""
    for specialization,skilllevel in list_of_people[person_index].skill_specialization_dict.items():
        description+=specialization+str(skilllevel)+","
    
    person_label = str(person_index)+": "+description[:-1]
    
    person_time_backlog_dict[person_label] = []
    for tick_index, journal_dict in list_of_people[person_index].work_journal_per_tick.items():
        #print(person_id, tick_index, journal_dict['number of processes in backlog='])
        person_time_backlog_dict[person_label].append(journal_dict['number of processes in backlog='])

In [ ]:
df_person_time_backlog = pandas.DataFrame(person_time_backlog_dict)
#df_person_time_backlog

In [ ]:
# https://seaborn.pydata.org/generated/seaborn.heatmap.html

seaborn.heatmap(df_person_time_backlog)
plt.title('backlog length per person')
plt.gca().set_xlabel('person', fontsize=15);
plt.gca().set_ylabel('time [ticks]', fontsize=15);

In [ ]:
for index,person in enumerate(list_of_people):
    print(index,person.skill_specialization_dict)

## process ID per person versus time

In [ ]:
person_time_processID_dict = {}
for person_id, person in enumerate(list_of_people):
    person_time_processID_dict[person_id] = []
    for tick, work_dict in person.work_journal_per_tick.items():
        #print(work_dict)
        person_time_processID_dict[person_id].append(work_dict['task']['process ID'])

In [ ]:
df_person_time_processID = pandas.DataFrame(person_time_processID_dict)
df_person_time_processID

In [ ]:
# https://seaborn.pydata.org/generated/seaborn.heatmap.html

seaborn.heatmap(df_person_time_processID)
plt.title('process ID')
plt.gca().set_xlabel('person', fontsize=15);
plt.gca().set_ylabel('time [ticks]', fontsize=15);

# ratio of the elapsed time for a process versus the number task for process.

In [ ]:
elapsed_process_timing = {}
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.max.html
for process_id in range(min(df_person_time_processID.min()),
                        max(df_person_time_processID.max())):
    # https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html
    list_of_ticks_containing_process = list(df_person_time_processID[
        df_person_time_processID==process_id].dropna(axis=0,how='all').index)
    
    assert(process_id==list_of_processes[process_id-1].list_of_tasks[0]['process ID'])
    
    elapsed_process_timing[process_id] = {'first seen': min(list_of_ticks_containing_process),
                                          'last seen': max(list_of_ticks_containing_process),
                                          'number of tasks': len(list_of_processes[process_id-1].list_of_tasks)}

In [ ]:
df_process_timing = pandas.DataFrame(elapsed_process_timing).T
df_process_timing['process actual duration'] = df_process_timing['last seen'] - df_process_timing['first seen']+1
df_process_timing['ratio of actual to minimum'] = df_process_timing['process actual duration']/df_process_timing['number of tasks']
df_process_timing

In [ ]:
# TODO: Heat map with axes of time and person index, 
# with the color being the status (working, idle, coordinating)

In [ ]:
# TODO: A pie chart of the named persona types like unicorn and specialist. 
# The biggest piece of pie will be the others

## task throughput vs time

In [ ]:
task_count_per_time_dict = {}
for person_id, person in enumerate(list_of_people):
    for tick, work_dict in person.work_journal_per_tick.items():
        if tick not in task_count_per_time_dict.keys():
            task_count_per_time_dict[tick] = 0
            
        if "task from" in work_dict.keys():
            #print(person_id, tick, work_dict["task from"])
            task_count_per_time_dict[tick] += 1

In [ ]:
plt.scatter(x=task_count_per_time_dict.keys(), 
            y=task_count_per_time_dict.values())
plt.ylabel('number of tasks\nstarted during this tick')
plt.xlabel('time [ticks]');

In [ ]:
plt.scatter(x=list(task_count_per_time_dict.keys()), 
            y=list(numpy.cumsum(list(task_count_per_time_dict.values()))),
            label="number of people="+str(number_of_people))

plt.ylabel('cumulative task count')
plt.xlabel('time [ticks]');
plt.legend();

# The reward for good work is more work

In [ ]:
people_versus_time_task_count={}
for person_id, person in enumerate(list_of_people):
    people_versus_time_task_count[person_id] = []
    completed_task_count = 0
    for tick, work_dict in person.work_journal_per_tick.items():
        if work_dict['outcome']=='task completed':
            completed_task_count+=1
        people_versus_time_task_count[person_id].append(completed_task_count)

In [ ]:
list_of_labels = []
list_of_task_counts = []

for person_index, completed_task_count in people_versus_time_task_count.items():
    description=""
    for specialization,skilllevel in list_of_people[person_index].skill_specialization_dict.items():
        description+=specialization+str(skilllevel)+","

    person_label = "person "+str(person_index)+": "+description[:-1]
    
    # these two lists are not used in this cell; they are for the stacked plot
    list_of_labels.append(person_label)
    list_of_task_counts.append(completed_task_count)
        
    plt.scatter(x=range(len(completed_task_count)),
                y=completed_task_count, 
                label=person_label)
    
# https://stackoverflow.com/a/4701285
plt.gca().legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.xlabel('time [ticks]')
plt.ylabel('cumulative count\nof completed tasks');
plt.title('social circle size ='+str(social_circle_size));

In [ ]:
for index,person in enumerate(list_of_people):
    print(index,person.skill_specialization_dict)

## work done per person

In [ ]:
# https://www.python-graph-gallery.com/251-stacked-area-chart-with-seaborn-style
# https://www.python-graph-gallery.com/250-basic-stacked-area-chart

# set seaborn style
seaborn.set_theme()
    
plt.stackplot(range(len(completed_task_count)),
              list_of_task_counts,
              labels=list_of_labels)
    
# https://stackoverflow.com/a/4701285
plt.gca().legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.xlabel('time [ticks]')
plt.ylabel('cumulative count\nof completed tasks');
plt.title('social circle size ='+str(social_circle_size));

## percentage stacked area chart

https://www.python-graph-gallery.com/255-percentage-stacked-area-chart

In [ ]:
task_dict = {}
for person_index, cumulative_task_count in enumerate(list_of_task_counts):
    task_dict[person_index] = cumulative_task_count
    
df_task_cumulative_count = pandas.DataFrame(task_dict)

In [ ]:
# https://stackoverflow.com/a/46738366

df_task_cumulative_percent = df_task_cumulative_count.divide(df_task_cumulative_count.sum(axis=1), axis=0)
  
#plot_me = df_task_cumulative_percent.T

#plot_me = df_task_cumulative_percent[df_task_cumulative_percent.columns[::-1]]

#plot_me[plot_me.columns[::-1]]

#df_to_plot = df_task_cumulative_percent.T


# https://stackoverflow.com/a/27817770

In [ ]:
plt.stackplot(range(len(completed_task_count)),
              #df_task_cumulative_percent[df_task_cumulative_percent.columns[::-1]].T,
              df_task_cumulative_percent.T,
              #plot_me.T,
              #df_task_cumulative_percent[df_task_cumulative_percent.columns[::-1]].T,
              labels=list_of_labels)
    
# https://stackoverflow.com/a/4701285
plt.gca().legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.xlabel('time [ticks]')
plt.ylabel('percent of\ncompleted tasks');
plt.title('social circle size ='+str(social_circle_size));

# TODO: https://en.wikipedia.org/wiki/Little%27s_law